In [92]:
%load_ext autoreload
%autoreload 2

import numpy as np
from cartpole.cartpole import Cartpole
import pickle, os

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [93]:
#load train/test data
cp = Cartpole() #use default config, pass different config file oth.
config_fn = './cartpole/config/default.p'

config_file = open(config_fn,'rb')
dataset_name, _, _ = pickle.load(config_file); config_file.close()

relative_path = os.getcwd()
dataset_fn = relative_path + '/cartpole/data/' + dataset_name

train_file = open(dataset_fn+'/train.p','rb')
# p_train, x_train, u_train, y_train, c_train, times_train = pickle.load(train_file)
train_data = pickle.load(train_file)
train_file.close()

test_file = open(dataset_fn+'/train.p','rb')
# p_test, x_test, u_test, y_test, c_test, times_test = pickle.load(test_file)
train_data = pickle.load(test_file)
test_file.close()

In [ ]:
#pre-processing

#some function here to convert p_test into a feature matrix, X
#some function here to convert y_test into strategy labels, y

In [ ]:
#training setup

#instantiate Solver object
#if first time, setup hyperparams + write out to a config file
#else, load hyperparams from config (e.g. network weights, lr, etc.)

In [ ]:
#training step

#either we can write a big loop here, or hide it in some Solver.train() method.

In [ ]:
#testing setp

#either we can write a big loop here, or hide it in some Solver.test() method.

In [ ]:
## dummy cell to test basic Cartpole functionality

xg = 0.1*np.random.randn(4); x0 = 0.1*np.random.randn(4) #dummy param values
params = {'x0': x0,'xg' : xg} #put them into a dict
strat = np.zeros((4,cp.N-1)).astype(int) #dummy strategy (all zeros)

feas_bin, optval_bin, time_bin, assigns_bin = cp.solve_micp(params) #solve problem exactly
feas, optval, time = cp.solve_pinned(params,strat)

# test MLOPT solver

In [94]:
from solvers.mlopt import MLOPT

system = 'cartpole'
prob_features = ['x0', 'xg']
mlopt_obj = MLOPT(system, cp, prob_features)

n_features = 8
mlopt_obj.construct_strategies(n_features, train_data)
print(mlopt_obj.n_strategies)

34


In [95]:
mlopt_obj.setup_network()
mlopt_obj.model

FFNet(
  (activation): ReLU()
  (layers): ModuleList(
    (0): Linear(in_features=8, out_features=32, bias=True)
    (1): Linear(in_features=32, out_features=32, bias=True)
    (2): Linear(in_features=32, out_features=32, bias=True)
    (3): Linear(in_features=32, out_features=34, bias=True)
  )
)

In [96]:
mlopt_obj.training_params['TRAINING_ITERATIONS'] = 7
mlopt_obj.train()
print(mlopt_obj.model_fn)

Done with epoch 0 in 0.0036318302154541016s
Done with epoch 1 in 0.003412008285522461s
Done with epoch 2 in 0.003379344940185547s
Done with epoch 3 in 0.003952503204345703s
Done with epoch 4 in 0.0033752918243408203s
Done with epoch 5 in 0.0033588409423828125s
Done with epoch 6 in 0.0033075809478759766s
Saved model at /home/acauligi/cs_234/project/mlopt-micp/mlopt_cartpole_20200618_1541.pt
Done training
/home/acauligi/cs_234/project/mlopt-micp/mlopt_cartpole_20200618_1541.pt


In [97]:
prob_params = {}
idx = 1

p_test = train_data[0]
for k in p_test.keys():
    prob_params[k] = p_test[k][idx]
prob_params

{'x0': array([ 0.09882732, -0.23948447, -0.7159997 ,  0.44798102]),
 'xg': array([0., 0., 0., 0.])}

In [98]:
prob_success, cost, total_time, n_evals = mlopt_obj.forward(prob_params)

if prob_success:
    print(cost)
else:
    print('Failed')

Failed


In [99]:
prob_success, true_cost, solve_time, optvals = mlopt_obj.problem.solve_micp(prob_params)

print(true_cost)

96.66295784630502


# test Regression solver

In [118]:
from solvers.regression import Regression

system = 'cartpole'
prob_features = ['x0', 'xg']
reg_obj = Regression(system, cp, prob_features)

n_features = 8
reg_obj.construct_strategies(n_features, train_data)
print(reg_obj.n_strategies)

34


In [119]:
reg_obj.setup_network()
reg_obj.model

FFNet(
  (activation): ReLU()
  (layers): ModuleList(
    (0): Linear(in_features=8, out_features=32, bias=True)
    (1): Linear(in_features=32, out_features=32, bias=True)
    (2): Linear(in_features=32, out_features=32, bias=True)
    (3): Linear(in_features=32, out_features=40, bias=True)
  )
)

In [120]:
reg_obj.training_params['TRAINING_ITERATIONS'] = 10
reg_obj.training_params['CHECKPOINT_AFTER'] = 200000
reg_obj.train()
print(reg_obj.model_fn)

Done with epoch 0 in 0.002160787582397461s
Done with epoch 1 in 0.0017833709716796875s
Done with epoch 2 in 0.0017423629760742188s
Done with epoch 3 in 0.0017457008361816406s
Done with epoch 4 in 0.0017313957214355469s
Done with epoch 5 in 0.00174713134765625s
Done with epoch 6 in 0.0018074512481689453s
Done with epoch 7 in 0.001741647720336914s
Done with epoch 8 in 0.001970529556274414s
Done with epoch 9 in 0.0017445087432861328s
Saved model at /home/acauligi/cs_234/project/mlopt-micp/mlopt_cartpole_20200618_1549.pt
Done training
/home/acauligi/cs_234/project/mlopt-micp/mlopt_cartpole_20200618_1549.pt


In [121]:
prob_params = {}
idx = 1

p_test = train_data[0]
for k in p_test.keys():
    prob_params[k] = p_test[k][idx]
prob_params

{'x0': array([ 0.09882732, -0.23948447, -0.7159997 ,  0.44798102]),
 'xg': array([0., 0., 0., 0.])}

In [122]:
prob_success, cost, total_time = reg_obj.forward(prob_params)

if prob_success:
    print(cost)
else:
    print('Failed')

Failed
